In [25]:
# import matching rows
path = "/scratchc/fmlab/zuberi01/phd/sequoia-pub/examples/matching_rows.csv"

import pandas as pd
df = pd.read_csv(path)
df.head()

wsi_file_name_col_name = "slide_id"
patient_id_col_name = "PatientID_SM"

# get all the column names that have a ":" in them from the df
cnv_col_names = [col for col in df.columns if ":" in col]

# drop all the columns that aren't the ones we specified above
df = df.drop(columns=[col for col in df.columns if col not in [wsi_file_name_col_name, patient_id_col_name] + cnv_col_names])

# add to all the csv_col_names in the df the "cnv_" prefix
df = df.rename(columns={col: "cnv_" + col for col in cnv_col_names})

new_csv = "matching_rows_sequoia.csv"
df.to_csv(new_csv, index=False)

In [26]:
# need to add the train,val,test column
# we want to add one for each split

# so let's get the splits and make a dictionary where the key is the wsi file name and the value is the split they are in

# get all the paths there are in phd/splits and the splits names are splits_0.csv, splits_1.csv, etc
all_splits_paths = "/scratchc/fmlab/zuberi01/phd/splits"
import os
split_csv_paths = [os.path.join(all_splits_paths, split) for split in os.listdir(all_splits_paths) if "splits" in split]
split_csv_paths = sorted(split_csv_paths)

all_splits_dicts = []
for split_csv_path in split_csv_paths:
    split_df = pd.read_csv(split_csv_path)

    # there are 4 columns of different sizes: 1st can be ignored, 2nd is train, 3rd is val, 4th is test
    train_split = split_df.iloc[:,1]
    val_split = split_df.iloc[:,2].dropna()
    test_split = split_df.iloc[:,3].dropna()

    # now we can make the dictionary
    split_dict = {}
    for wsi in train_split:
        split_dict[wsi] = "train"   
    for wsi in val_split:
        split_dict[wsi] = "val"
    for wsi in test_split:
        split_dict[wsi] = "test"

    all_splits_dicts.append(split_dict)

# import the new_csv and loop through the wsi_file_name_col_name column and add the split column called split_x where x is the index of the split
df = pd.read_csv(new_csv)
for i, split_dict in enumerate(all_splits_dicts):
    split_col_name = f"split_{i}"
    df[split_col_name] = df[wsi_file_name_col_name].map(split_dict)

df.to_csv(new_csv, index=False)



In [27]:
df['tcga_project'] = 'h5_files'
df.to_csv(new_csv, index=False)

In [21]:
path_h5 = "/scratchc/fmlab/zuberi01/masters/saved_patches/40x_400/features2/h5_files/001H056696 PR1 HIN 042 B B2506 1.h5"
import h5py
import numpy

with h5py.File(path_h5, 'r') as f:
    # List all groups
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]

    # Get the data
    data = list(f[a_group_key])
    print(data)

    # Get the metadata
    metadata = f[a_group_key].attrs
    print(metadata)

    # Get the image
    image = f[a_group_key]["image"]
    print(image)



Keys: <KeysViewHDF5 ['coords', 'features']>
[array([ 3648, 33088], dtype=int32), array([ 3648, 33488], dtype=int32), array([ 3648, 33888], dtype=int32), array([ 3648, 34288], dtype=int32), array([ 3648, 34688], dtype=int32), array([ 3648, 35088], dtype=int32), array([ 4048, 33088], dtype=int32), array([ 4048, 33488], dtype=int32), array([ 4048, 33888], dtype=int32), array([ 4048, 34288], dtype=int32), array([ 4048, 34688], dtype=int32), array([ 4048, 35088], dtype=int32), array([ 4448, 33888], dtype=int32), array([ 4448, 34288], dtype=int32), array([ 4448, 34688], dtype=int32), array([ 4848, 33488], dtype=int32), array([ 4848, 33888], dtype=int32), array([ 4848, 34288], dtype=int32), array([ 4848, 34688], dtype=int32), array([ 5248, 33488], dtype=int32), array([ 5248, 33888], dtype=int32), array([ 5248, 34288], dtype=int32), array([ 5248, 34688], dtype=int32), array([ 5648, 33488], dtype=int32), array([ 5648, 33888], dtype=int32), array([ 5648, 34288], dtype=int32), array([ 5648, 34688

ValueError: Field names only allowed for compound types